# Atom feed of tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least three times as atom feed, grouped by subject area.

In [1]:
import sys
sys.path.append('../')
!{sys.executable} -m pip install lxml
import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator
import lxml

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
date = str(date.today().strftime('%Y-%m-%d'))
subject_areas = ['covid', 'medical_health-sciences', 'bioinformatics', 'biochemistry_cell-biology', 'cancer', 'ecology', 'evolutionary-biology', 'genetics', 'microbiology', 'neurosciences', 'physiology', 'plant-biology', 'zoology','other-biological-sciences']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + date + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
print('COVID: ' + str(len(subsets['covid'])))
subsets['medical_health-sciences'] = preprints[((preprints['archive'] == 'medRxiv')  | (preprints['subject-area'] == 'Pathology') | (preprints['subject-area'] == 'Pharmacology and Toxicology'))  & (preprints['covid'] == False)]
print('Medical and Health Sciences: ' + str(len(subsets['medical_health-sciences'])))
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
print('Bioinformatics: ' + str(len(subsets['bioinformatics'])))
subsets['biochemistry_cell-biology'] = preprints[((preprints['subject-area'] == 'Biochemistry') | (preprints['subject-area'] == 'Cell Biology') | (preprints['subject-area'] == 'Developmental Biology') | (preprints['subject-area'] == 'Synthetic Biology') | (preprints['subject-area'] == 'Systems Biology') | (preprints['subject-area'] == 'Molecular Biology')) & (preprints['covid'] == False)]
print('Biochemistry and Cell Biology: ' + str(len(subsets['biochemistry_cell-biology'])))
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['cancer'] = preprints[((preprints['subject-area'] == 'Cancer Biology') | (preprints['subject-area'] == 'Hematology') | (preprints['subject-area'] == 'Oncology')) & (preprints['covid'] == False)]
print('Cancer: ' + str(len(subsets['cancer'])))
print('Ecology: ' + str(len(subsets['ecology'])))
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
print('Evolutionary Biology: ' + str(len(subsets['evolutionary-biology'])))
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') | (preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
print('Genetics: ' + str(len(subsets['genetics'])))
subsets['microbiology'] = preprints[((preprints['subject-area'] == 'Microbiology') | (preprints['subject-area'] == 'Immunology')) & (preprints['covid'] == False)]
print('Microbiology: ' + str(len(subsets['microbiology'])))
subsets['neurosciences'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
print('Neurosciences: ' + str(len(subsets['neurosciences'])))
subsets['physiology'] = preprints[(preprints['subject-area'] == 'Physiology') | (preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
print('Physiology: ' + str(len(subsets['physiology'])))
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
print('Plant Biology: ' + str(len(subsets['plant-biology'])))
subsets['zoology'] = preprints[(preprints['subject-area'] == 'Zoology') | (preprints['subject-area'] == 'Animal Behavior and Cognition') & (preprints['covid'] == False)]
print('Zoology: ' + str(len(subsets['zoology'])))
subsets['other-biological-sciences'] = preprints[(preprints['archive'] != 'medRxiv') & (preprints['subject-area'] != 'Animal Behavior and Cognition') & (preprints['subject-area'] != 'Bioinformatics') & (preprints['subject-area'] != 'Biochemistry') & (preprints['subject-area'] != 'Biophysics') & (preprints['subject-area'] != 'Cancer Biology') & (preprints['subject-area'] != 'Cell Biology') & (preprints['subject-area'] != 'Developmental Biology') & (preprints['subject-area'] != 'Ecology') & (preprints['subject-area'] != 'Evolutionary Biology') & (preprints['subject-area'] != 'Genetics') & (preprints['subject-area'] != 'Genomics') & (preprints['subject-area'] != 'Immunology') & (preprints['subject-area'] != 'Microbiology') & (preprints['subject-area'] != 'Molecular Biology') & (preprints['subject-area'] != 'Neuroscience') & (preprints['subject-area'] != 'Physiology') & (preprints['subject-area'] != 'Plant Biology') & (preprints['subject-area'] != 'Synthetic Biology') & (preprints['subject-area'] != 'Systems Biology') & (preprints['subject-area'] != 'Zoology') & (preprints['covid'] == False)]
print('Other Biological Sciences: ' + str(len(subsets['other-biological-sciences'])))
subsets['microbiology'].head(50)

COVID: 19
Medical and Health Sciences: 1
Bioinformatics: 6
Biochemistry and Cell Biology: 26
Cancer: 0
Ecology: 2
Evolutionary Biology: 12
Genetics: 9
Microbiology: 22
Neurosciences: 24
Physiology: 6
Plant Biology: 10
Zoology: 3
Other Biological Sciences: 3


,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
15,10.1101/2021.12.01.469893,32,bioRxiv,Immunology,False,Cellular selectivity of STING stimulation dete...,"[{'name': 'Bakhos Jneid'}, {'name': 'Aurore Bo...",<p>T cells that recognize tumor antigens are c...,2021-12-01
31,10.1101/2021.12.02.470747,14,bioRxiv,Microbiology,False,An ecological perspective on microbial genes o...,"[{'name': 'Hannah Holland-Moritz'}, {'name': '...","<p>Genes that remain hypothetical, uncharacter...",2021-12-02
41,10.1101/2021.12.01.470848,10,bioRxiv,Microbiology,False,Redox-active secondary metabolites act as inte...,"[{'name': 'Lucas A. Meirelles'}, {'name': 'Dia...",<p>Bacterial opportunistic pathogens make a wi...,2021-12-02
47,10.1101/2021.11.30.470380,9,bioRxiv,Microbiology,False,Communication determines population-level fitn...,"[{'name': 'Benedikt K. Steinfeld'}, {'name': '...",<p>Bacterial populations frequently encounter ...,2021-11-30
48,10.1101/2021.11.29.470317,9,bioRxiv,Microbiology,False,A positive feedback loop mediates crosstalk be...,"[{'name': 'Caia Dominicus'}, {'name': 'Stephan...",<p>Fundamental processes of obligate intracell...,2021-11-30
60,10.1101/2021.11.29.469274,7,bioRxiv,Microbiology,False,"Plasmodiophora brassicae in Mexico, from anecd...","[{'name': 'Legnara Padrón-Rodríguez'}, {'name'...","<p>For years, the presence of clubroot disease...",2021-11-30
68,10.1101/2021.12.01.470831,6,bioRxiv,Microbiology,False,The AdcACB/AdcAII system is Essential for Zinc...,"[{'name': 'Ling Ning Lam'}, {'name': 'Debra N....",<p>Bacterial pathogens require a variety of mi...,2021-12-01
70,10.1101/2021.12.01.470493,6,bioRxiv,Immunology,False,Tertiary lymphoid structures induced by CXCL13...,"[{'name': 'Masayo Ukita'}, {'name': 'Junzo Ham...",<sec><title>Background</title><p>Tertiary lymp...,2021-12-01
74,10.1101/2021.12.01.470877,5,bioRxiv,Microbiology,False,Dynamic utilization of low-molecular-weight or...,"[{'name': 'K. Taylor Cyle'}, {'name': 'Annalei...","<p>Constantly in flux, low-molecular-weight or...",2021-12-02
81,10.1101/2021.12.03.471163,5,bioRxiv,Microbiology,False,Modelling of filamentous phage-induced antibio...,"[{'name': 'Maria Tessel van Rossem'}, {'name':...",<p>Filamentous molecules tend to spontaneously...,2021-12-03


In [3]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular {subject_area.replace("-", " ").replace("_", " and ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')